# Experimental API Download to XLSX

This script is a working prototype of XLSX conversion to/from the API, but is a pretty early prototype. I expect to do more work around the ID handling and generally making these forms work more like the XLSForm spec.

This script uses the newly-created form APIs to pull in all form details, based on the name to search

In [ ]:
import http.client
import json
from types import SimpleNamespace
import pandas as pd
from urllib.parse import quote
pd.set_option('display.max_colwidth', None)
!pip install xlsxwriter

# Org-specific stuff

In [ ]:
url_to_query = "twapi-demo-dev-ed.develop.my.salesforce.com"
# This is the name of the form to download from salesforce
form_name_to_download = 'Proof of Concept'
# This is the GDrive folder to place the finished file (make sure you have a folder in GDrie that lines up)
gdrive_folder = "PFI TW API/"
# This is the name of the file to save
fileName = 'proof-of-concept-before-translation.xlsx'

# Just run all the cells below this, only change them if you know what you're doing/enjoy frustration

# Login to GDrive

In [ ]:
# Prepare Google Drive
from google.colab import drive
# Mount Google Drive so we can pull in files
drive.mount('/content/drive')
# Setup some directory names (helps to keep work separate)
workingDirectory = '/content/drive/MyDrive/' + gdrive_folder

# Get all form components and save in excel

## Get the Form ID, Form Version ID and Change Log from the form name

Assume this only works for 1 form at a time, they can be squished in a separate script

In [ ]:
def get_oauth_token_from_login():
    conn = http.client.HTTPSConnection(url_to_query)
    payload = ''
    headers = {}
    #NOTE: This is bad security practice, normally we would want these login details to be hidden so they don't get backed up to git
    #For simplicity of this demo, we're exposing details here
    #TODO: before production usage, separate these variables into a JSON file that's loaded from Google Drive 
    login_url = "/services/oauth2/token?grant_type=password&client_id=xxx&client_secret=yyy&username=www&password=aaa"
    conn.request("POST",  login_url, payload, headers)
    res = conn.getresponse()
    data = res.read()
    decoded_form_data = data.decode("utf-8")
    data_obj = json.loads(decoded_form_data, object_hook=lambda d: SimpleNamespace(**d))
    print(data_obj)
    return "OAuth " + data_obj.access_token


In [ ]:
auth_header = get_oauth_token_from_login()
print(auth_header)

In [ ]:
def get_python_obj_from_json_web_call(endpoint_to_hit):
    conn = http.client.HTTPSConnection(url_to_query)
    payload = ''
    headers = {
      'Authorization': auth_header,
      'Content-Type': 'application/json',
    }
    conn.request("GET",  endpoint_to_hit, payload, headers)
    res = conn.getresponse()
    data = res.read()
    decoded_form_data = data.decode("utf-8")
    data_obj = json.loads(decoded_form_data, object_hook=lambda d: SimpleNamespace(**d))
    return data_obj

In [ ]:
def get_pandas_dataframe_from_json_web_call(endpoint_to_hit):
    conn = http.client.HTTPSConnection(url_to_query)
    payload = ''
    headers = {
      'Authorization': auth_header,
      'Content-Type': 'application/json',
    }
    conn.request("GET",  endpoint_to_hit, payload, headers)
    res = conn.getresponse()
    data = res.read()
    decoded_form_data = data.decode("utf-8")
    data_obj = json.loads(decoded_form_data)
    records_dataframe = pd.json_normalize(data_obj, record_path =['records'])
    return records_dataframe

In [ ]:
def upload_payload_to_url(endpoint_to_upload, payload):
      conn = http.client.HTTPSConnection(url_to_query)
      headers = {
        'Authorization': auth_header,
        'Content-Type': 'application/json',
      }
      conn.request("PUT", endpoint_to_upload, payload, headers)
      res = conn.getresponse()
      data = res.read()
      print(data.decode("utf-8"))

In [ ]:
def get_version_changelog_from_form_name(form_name_to_download):
    form_name_urlsafe = quote(form_name_to_download, safe='/')
    form_endpoint = "/services/apexrest/formdata/v1?objectType=GetFormData&name=" + form_name_urlsafe
    form_dataframe = get_pandas_dataframe_from_json_web_call(form_endpoint)
    # TODO - add better error if unable to reach server/auth is out of date
    try:
      form_id = form_dataframe.id[0]
      form_external_id = form_dataframe.externalId[0]
    except: 
      print('No form matches that name')
      return '', '', '','', None
    # Note - this script assumes there is only 1 form matching the name
    json_form_version = form_dataframe.formVersion[0]
    form_version_string = str(json_form_version[0]).replace('\'','"')
    form_version_json_obj = json.loads(form_version_string)
    form_version_id = form_version_json_obj['versionid']
    changelog_number = form_version_json_obj['changeLogNumber']
    print('Form Version ID: ',form_version_id, ' Form ID: ', form_id, ' Changelog: ', changelog_number, ' externalID: ',form_external_id)
    return form_version_id, changelog_number, form_id, form_external_id, form_dataframe

In [ ]:
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_download)

In [ ]:
form_dataframe

## Get all Questions

In [ ]:
question_endpoint = "/services/apexrest/questiondata/v1?objectType=GetQuestionData&formVersionId=" + form_version_id
question_dataframe = pd.DataFrame(columns=['externalId', 'id', 'name', 'caption', 'cascadingLevel',\
       'cascadingSelect', 'controllingQuestion', 'displayRepeatSectionInTable',\
       'dynamicOperation', 'dynamicOperationTestData', 'dynamicOperationType',\
       'exampleOfValidResponse', 'form', 'formVersion', 'hidden', 'maximum',\
       'minimum', 'parent', 'position', 'previousVersion', 'printAnswer',\
       'repeatSourceValue', 'repeatTimes', 'required', 'responseValidation',\
       'showAllQuestionOnOnePage', 'skipLogicBehavior', 'skipLogicOperator',\
       'hint', 'testDynamicOperation', 'type', 'useCurrentTimeAsDefault',\
       'changeLogNumber', 'options'])
question_dataframe = pd.concat([question_dataframe,get_pandas_dataframe_from_json_web_call(question_endpoint)])

In [ ]:
#Iterate all questions that have options and create a new dataframe that has just the options
options_dataframe = pd.DataFrame(columns=["externalId" , "id" , "name" , "position" , "caption","questionId" ])
for index, frame in question_dataframe.iterrows():
    if (frame.options):
      questionId = frame.id
      individual_option_df = pd.read_json(str(frame.options).replace('\'','"'))
      individual_option_df['questionId'] = questionId
      options_dataframe = pd.concat([individual_option_df,options_dataframe])

In [ ]:
questions_without_options = question_dataframe.loc[:, question_dataframe.columns != 'options']

In [ ]:
parentLookup = questions_without_options[questions_without_options['parent'] == ""][['position','name','id']].rename(columns={'position':'parentPosition','name':'parentName','id':'parentId'})

In [ ]:
questions_with_order = questions_without_options.copy()

In [ ]:
hackyMultiplier = 10000 #arbitrarily large hacky multiplier
questions_with_order['formOrder'] = questions_with_order.apply(lambda x: int(parentLookup[parentLookup['parentId'] == x['parent']]['parentPosition'].iloc[0]) * hackyMultiplier + int(x['position'])  if x['parent'] != "" else int(x['position']) * hackyMultiplier, axis =1 )

In [ ]:
questions_without_options = questions_with_order.sort_values(by=['formOrder']).drop(columns=['formOrder'])

## Get all Form/Field Mappings

In [ ]:
field_mapping_endpoint = "/services/apexrest/formmappingdata/v1?objectType=GetFormMappingData&formVersionId=" + form_version_id
field_mapping_dataframe = pd.DataFrame(columns = ['externalId', 'id', 'name', 'form', 'formVersion',\
       'formVersionMappingField', 'mobileUserField', 'objectApiName',\
       'formMappingField', 'isReference', 'matchingField', 'repeat',\
       'submissionAPIField', 'changeLogNumber', 'questionMappings'])
field_mapping_dataframe = pd.concat([field_mapping_dataframe,get_pandas_dataframe_from_json_web_call(field_mapping_endpoint)])

In [ ]:
#Iterate all form mappings that have question mappings and create a new dataframe that has just the question mappings
question_mapping_dataframe = pd.DataFrame(columns=["externalId", "name", "id", "fieldAPIName","isBroken","question","scoringGroup","field_mapping_id"])
for index, frame in field_mapping_dataframe.iterrows():
    if (frame.questionMappings):
      print(str(frame.questionMappings).replace('\'','"'))
      field_mapping_id = frame.id
      #JSON is case-sensitive, python apparently converts it into uppercase
      individual_question_mapping_df = pd.read_json(str(frame.questionMappings).replace('\'','"').replace("True","true").replace("False","false"))
      individual_question_mapping_df['field_mapping_id'] = field_mapping_id
      question_mapping_dataframe = pd.concat([individual_question_mapping_df,question_mapping_dataframe])

In [ ]:
field_mapping_without_questions = field_mapping_dataframe.loc[:, field_mapping_dataframe.columns != 'questionMappings']

## Get All Skip Logic

In [ ]:
skip_logic_endpoint = "/services/apexrest/skiplogicdata/v1?objectType=GetSkipLogicData&formVersionId=" + form_version_id
skip_logic_dataframe = pd.DataFrame(columns=["externalId" ,"id" ,"negate" ,"skipValue" ,"condition" ,"parentQuestion" ,"sourceQuestion" ,"form" ,"formVersion" ,"changeLogNumber"])
skip_logic_dataframe = pd.concat([skip_logic_dataframe, get_pandas_dataframe_from_json_web_call(skip_logic_endpoint)])

## Get All Object Relationship Mapping

In [ ]:
orm_endpoint = "/services/apexrest/objectrelationshipmappingdata/v1?objectType=GetObjectRelationshipMappingData&formVersionId=" + form_version_id
orm_dataframe = pd.DataFrame(columns=["externalId" ,"id" ,"name" ,"fieldApiName" ,"parentSurveyMapping" ,"childSurveyMapping" ,"formVersion" ,"changeLogNumber"])
orm_dataframe = pd.concat([orm_dataframe, get_pandas_dataframe_from_json_web_call(orm_endpoint)])

In [ ]:
orm_dataframe

## Replace IDs
Replace internal salesforce IDs + actual externalIds with computed external IDs

In [ ]:
form_dataframe_id_replaced = form_dataframe.copy()
# default to english if form doesn't already contain it
# assume that only 1 form is present, so this should only be 1 row
external_id_in_salesforce = form_dataframe_id_replaced['externalId'][0]
if (not external_id_in_salesforce):
    external_id_in_salesforce = form_dataframe_id_replaced['name'][0]
taro_id_full = external_id_in_salesforce
taro_id_parent_form = external_id_in_salesforce
taro_language = 'en'
if ('::' in external_id_in_salesforce):
    taro_id_parent_form = external_id_in_salesforce.split('::')[0]
    taro_language = external_id_in_salesforce.split('::')[1]

print(taro_id_parent_form)
print(taro_language)
#remove taroId column, just use name for this
#form_dataframe_id_replaced['taroId'] = taro_id_parent_form + '::' + taro_language
#form_dataframe_id_replaced['changeLog'] = changelog_number
form_dataframe_id_replaced = form_dataframe_id_replaced.drop(columns=['id','externalId','formVersion'])

In [ ]:
questions_without_options_id_replaced = questions_without_options.copy()
questions_without_options_id_replaced = questions_without_options_id_replaced.rename(columns={'caption':'caption::'+taro_language})
questions_id_lookup = questions_without_options_id_replaced[['id','name']].rename(columns={'name':'questionName','id':'questionId'})
questions_without_options_id_replaced = questions_without_options_id_replaced.merge(questions_id_lookup,how="left",left_on="parent",right_on="questionId").rename(columns={'questionName':'parentName'})
#remove taroId column, just use name for this
#questions_without_options_id_replaced['taroId'] = questions_without_options_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else x['name'], axis=1)
questions_without_options_id_replaced.drop(columns=['externalId'],inplace=True)
questions_without_options_id_replaced = questions_without_options_id_replaced.drop(columns=['id','changeLogNumber','form','formVersion','questionId','parent'])

In [ ]:
options_dataframe_id_replaced = options_dataframe.copy()
options_dataframe_id_replaced = options_dataframe_id_replaced.rename(columns={'caption':'caption::'+taro_language})
options_dataframe_id_replaced = options_dataframe_id_replaced.merge(questions_id_lookup,how='left',on='questionId')
#remove taroId column, just use name for this
# if (not options_dataframe_id_replaced.empty):
#     options_dataframe_id_replaced['taroId'] = options_dataframe_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else x['name'], axis=1)
# else:
#     options_dataframe_id_replaced['taroId'] = None
options_dataframe_id_replaced.drop(columns=['externalId'],inplace=True)
options_dataframe_id_replaced = options_dataframe_id_replaced.drop(columns=['id','questionId'])

In [ ]:
field_mapping_without_questions_id_replaced = field_mapping_without_questions.copy()
field_mapping_without_questions_id_replaced = field_mapping_without_questions_id_replaced.merge(questions_id_lookup,how="left",left_on="repeat",right_on="questionId")
field_mapping_id_lookup = field_mapping_without_questions_id_replaced[['id','name']].rename(columns={'id':'fieldMappingId','name':'fieldMappingName'})
#remove taroId column, just use name for this
# if (not field_mapping_without_questions_id_replaced.empty):
#     field_mapping_without_questions_id_replaced['taroId'] = field_mapping_without_questions_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else x['name'], axis=1)
# else:
#     field_mapping_without_questions_id_replaced['taroId'] = None
field_mapping_without_questions_id_replaced.drop(columns=['externalId'],inplace=True)
field_mapping_without_questions_id_replaced = field_mapping_without_questions_id_replaced.drop(columns=['id','form','formVersion','changeLogNumber','repeat','questionId'])
field_mapping_without_questions_id_replaced = field_mapping_without_questions_id_replaced.rename(columns={'questionName':'repeatQuestionName'}).fillna('')

In [ ]:
question_mapping_dataframe_id_replaced = question_mapping_dataframe.copy()
question_mapping_dataframe_id_replaced = question_mapping_dataframe_id_replaced.merge(field_mapping_id_lookup,how="left",left_on="field_mapping_id",right_on="fieldMappingId")
question_mapping_dataframe_id_replaced = question_mapping_dataframe_id_replaced.merge(questions_id_lookup,left_on='question',right_on = 'questionId')
#remove taroId column, just use name for this
# if (not question_mapping_dataframe_id_replaced.empty):
#     question_mapping_dataframe_id_replaced['taroId'] = question_mapping_dataframe_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else x['name'], axis=1)
# else:
#     question_mapping_dataframe_id_replaced['taroId'] = None
question_mapping_dataframe_id_replaced.drop(columns=['externalId'],inplace=True)
question_mapping_dataframe_id_replaced = question_mapping_dataframe_id_replaced.drop(columns=['id','question','fieldMappingId','field_mapping_id','questionId'])

In [ ]:
skip_logic_dataframe_id_replaced = skip_logic_dataframe.copy()
skip_logic_dataframe_id_replaced = skip_logic_dataframe_id_replaced.merge(questions_id_lookup,left_on='sourceQuestion',right_on='questionId').rename(columns={'questionName':'sourceQuestionName'}).drop(columns=['questionId'])

skip_logic_dataframe_id_replaced = skip_logic_dataframe_id_replaced.merge(questions_id_lookup,left_on='parentQuestion',right_on='questionId').rename(columns={'questionName':'parentQuestionName'}).drop(columns=['questionId'])
#create a fictitious name column (external ID if it exists, join column if not)
if (not skip_logic_dataframe_id_replaced.empty):
    skip_logic_dataframe_id_replaced['name'] = skip_logic_dataframe_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else str(x['sourceQuestion']) + str(x['parentQuestion']), axis=1)
else:
    skip_logic_dataframe_id_replaced['name'] = None
skip_logic_dataframe_id_replaced.drop(columns=['externalId'],inplace=True)
#skip_logic_dataframe_id_replaced = skip_logic_dataframe_id_replaced.rename(columns = {'externalId':'taroId'})
skip_logic_dataframe_id_replaced = skip_logic_dataframe_id_replaced.drop(columns=['id','parentQuestion','sourceQuestion','form','formVersion','changeLogNumber'])


In [ ]:
orm_dataframe_id_replaced = orm_dataframe.copy()
orm_dataframe_id_replaced = orm_dataframe_id_replaced.merge(field_mapping_id_lookup.rename(columns={'fieldMappingName':'parentSurveyName'}),how='left',left_on='parentSurveyMapping',right_on='fieldMappingId').drop(columns=['fieldMappingId'])
orm_dataframe_id_replaced = orm_dataframe_id_replaced.merge(field_mapping_id_lookup.rename(columns={'fieldMappingName':'childSurveyName'}),how='left',left_on='childSurveyMapping',right_on='fieldMappingId').drop(columns=['fieldMappingId'])
#remove taroId column, just use name for this
# if (not orm_dataframe_id_replaced.empty):
#     orm_dataframe_id_replaced['taroId'] = orm_dataframe_id_replaced.apply(lambda x: str(x['externalId']) if x['externalId'] else x['name'], axis=1)
# else: 
#   orm_dataframe_id_replaced['taroId'] = None
orm_dataframe_id_replaced.drop(columns=['externalId'],inplace=True)
orm_dataframe_id_replaced = orm_dataframe_id_replaced.drop(columns=['id','parentSurveyMapping','childSurveyMapping','formVersion','changeLogNumber'])


# Write an Excel Sheet

In [ ]:
form_dataframe_id_replaced

In [ ]:
writer = pd.ExcelWriter('/tmp/test.xlsx',engine='xlsxwriter')
workbook=writer.book

# https://datascience.stackexchange.com/questions/46437/how-to-write-multiple-data-frames-in-an-excel-sheet
# form_dataframe
# questions_without_options
# options_dataframe
# question_mapping_dataframe
# field_mapping_without_questions
# skip_logic_dataframe
# orm_dataframe


form_dataframe_id_replaced.to_excel(writer,sheet_name='Forms',startrow=1 , startcol=0,index=False)
questions_without_options_id_replaced.to_excel(writer,sheet_name='Questions',startrow=1 , startcol=0,index=False)
options_dataframe_id_replaced.to_excel(writer,sheet_name='Options',startrow=1 , startcol=0,index=False)
question_mapping_dataframe_id_replaced.to_excel(writer,sheet_name='Question_Mappings',startrow=1 , startcol=0,index=False)
field_mapping_without_questions_id_replaced.to_excel(writer,sheet_name='Field_Mappings',startrow=1 , startcol=0,index=False)
skip_logic_dataframe_id_replaced.to_excel(writer,sheet_name='Skip_Logic',startrow=1 , startcol=0,index=False)
orm_dataframe_id_replaced.to_excel(writer,sheet_name='Object_Relationship_Mappings',startrow=1 , startcol=0,index=False)
writer.save()

In [ ]:
!cp /tmp/test.xlsx "{workingDirectory + fileName}"